## Building Agents 🤖 with RAGs 📀 and Tools 🛠️ support with Local LLM 🧠

<img src="./Images/agents.png" width="800" height="500" style="display: block; margin: auto;">

In [3]:
from dotenv import load_dotenv
from langchain_ollama import ChatOllama

load = load_dotenv('./../.env')


llm = ChatOllama(
    base_url="http://localhost:11434",
    model = "qwen2.5:latest",
    temperature=0.5,
    max_tokens = 250
)

#### Playwright Tool Code

In [4]:
from langchain_community.agent_toolkits import PlayWrightBrowserToolkit
from langchain_community.tools.playwright.utils import (
    create_async_playwright_browser
)
import nest_asyncio

nest_asyncio.apply()

In [ ]:
async_browser = create_async_playwright_browser()
toolkit = PlayWrightBrowserToolkit.from_browser(async_browser=async_browser)
tools = toolkit.get_tools()
tools

[ClickTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/Users/karthik/Library/Caches/ms-playwright/chromium-1155/chrome-mac/Chromium.app/Contents/MacOS/Chromium> version=133.0.6943.16>),
 NavigateTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/Users/karthik/Library/Caches/ms-playwright/chromium-1155/chrome-mac/Chromium.app/Contents/MacOS/Chromium> version=133.0.6943.16>),
 NavigateBackTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/Users/karthik/Library/Caches/ms-playwright/chromium-1155/chrome-mac/Chromium.app/Contents/MacOS/Chromium> version=133.0.6943.16>),
 ExtractTextTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/Users/karthik/Library/Caches/ms-playwright/chromium-1155/chrome-mac/Chromium.app/Contents/MacOS/Chromium> version=133.0.6943.16>),
 ExtractHyperlinksTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/Users/karthik/Library/Caches/ms-p

### Initialize Vector DB and Retreiver along with Embedding

##### Embedding

In [6]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="llama3.2:latest")

#### Chroma DB

In [ ]:
from langchain_chroma import Chroma
vector_store = Chroma(persist_directory='./../Section5_RAGWithDocuments/chroma_langchain_db', embedding_function=embeddings)

result = vector_store.similarity_search("What is Bias testing", k=3)

for doc in result:
    print(doc.page_content)


60 CHAPTER 4. LOGICAL REASONING CORRECTNESS
the following challenges: 1) If an LLM concludes correctly, it is unclear
whether the response stems from reasoning or merely relies on simple
heuristics such as memorization or word correlations (e.g., “dry floor”
is more likely to correlate with “playing football”). 2) If an LLM
fails to reason correctly, it is not clear which part of the reasoning
process it failed (i.e., inferring not raining from floor being dry or
inferring playing football from not raining). 3) There is a lack of
a system that can organize such test cases to cover all other formal
reasoning scenarios besides implication, such as logical equivalence
(e.g., If A then B, if B then A; therefore, A if and only if B). 4)
Furthermore, understanding an LLM’s performance on such test cases
provides little guidance on improving the reasoning ability of the
LLM. To better handle these challenges, a well-performing testing
Li Li, Xiapu Luo, David Lo, John Grundy, and Haoyu Wang,
“

#### Retriever from Vector Stores

In [8]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs = {"k": 1}
)

#### Creating a tool to retreive the data from Vector stores

In [15]:
from langchain.tools import tool

@tool
def bias_detection(query: str) -> str:
    """
    What kind of Bias is mentioned in this article
    You must use this tool for Bias related testing in LLM (Large Language Model)
    
    Args:
        query: The search query
    """
    
    return retriever.invoke(query)

##### Add the bias detection tool in the array of tools

In [16]:
tools.append(bias_detection)

tools

[ClickTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/Users/karthik/Library/Caches/ms-playwright/chromium-1155/chrome-mac/Chromium.app/Contents/MacOS/Chromium> version=133.0.6943.16>),
 NavigateTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/Users/karthik/Library/Caches/ms-playwright/chromium-1155/chrome-mac/Chromium.app/Contents/MacOS/Chromium> version=133.0.6943.16>),
 NavigateBackTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/Users/karthik/Library/Caches/ms-playwright/chromium-1155/chrome-mac/Chromium.app/Contents/MacOS/Chromium> version=133.0.6943.16>),
 ExtractTextTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/Users/karthik/Library/Caches/ms-playwright/chromium-1155/chrome-mac/Chromium.app/Contents/MacOS/Chromium> version=133.0.6943.16>),
 ExtractHyperlinksTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/Users/karthik/Library/Caches/ms-p

### Creating Agent code to call the RAG and Tools

In [17]:
from langchain.agents import initialize_agent, AgentType

agent = initialize_agent(
    tools= tools,
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

query = "Extract the page and check if there is bias in the article https://www.thetimes.com/uk/media/article/bbc-coverage-hostile-israel-jewish-groups-ftkcblhs5"

result = await agent.arun(query)

print(result)



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "navigate_browser",
  "action_input": {
    "url": "https://www.thetimes.com/uk/media/article/bbc-coverage-hostile-israel-jewish-groups-ftkcblhs5"
  }
}
```

Observation: Navigating to https://www.thetimes.com/uk/media/article/bbc-coverage-hostile-israel-jewish-groups-ftkcblhs5 returned status code 200
Thought:Action:
```
{
  "action": "extract_text",
  "action_input": {}
}
```
Observation: BBC coverage ‘institutionally hostile’ to Israel, say Jewish groups We haven't been able to take payment You must update your payment details via My Account or by clicking update payment details to keep your subscription. Update payment details Act now to keep your subscription We've tried to contact you several times as we haven't been able to take payment. You must update your payment details via My Account or by clicking update payment details to keep your subscription. Update payment details Your subscription is due to terminate W